In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, precision_recall_curve

import tensorflow as tf
import keras 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Conv2D, MaxPooling2D

np.random.seed(42)
tf.random.set_seed(42)

from preprocessing.getdata import *

In [2]:
df = get_csvdata()
dfTrain, dfTest, y_train, y_test = train_test_split(df, df['CDR'], stratify = df['CDR'], random_state=42)
N=5
X_train = get_slices(dfTrain['ID'], N=N, d=2, dim=2, m=90)
y_train = y_train.repeat(1+2*N)
X_test = get_slices(dfTest['ID'], dim=2, m=90)

In [3]:
X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

In [4]:
HEIGHT = X_train.shape[1]
WIDTH = X_train.shape[2]

In [5]:
INPUT_SHAPE = (HEIGHT, WIDTH, 3)
b_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

model = Sequential()
model.add(InputLayer(input_shape=INPUT_SHAPE))
model.add(b_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

# Defining optimizer and learning rate
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.00001,
    decay_steps=10000,
    decay_rate=1,
    staircase=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

model_history = model.fit(X_train, y_train, epochs=30, shuffle=True, validation_data=(X_test, y_test))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-23 20:56:01.356819: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-23 20:56:01.357811: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2022-08-23 20:56:03.992703: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-23 20:56:04.709519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - ETA: 0s - loss: 0.6961 - accuracy: 0.5314

2022-08-23 20:57:35.183091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 92s 2s/step - loss: 0.6961 - accuracy: 0.5314 - val_loss: 0.6729 - val_accuracy: 0.5217
Epoch 2/30
47/47 [==============================] - 93s 2s/step - loss: 0.6044 - accuracy: 0.6725 - val_loss: 0.6832 - val_accuracy: 0.6087
Epoch 3/30
47/47 [==============================] - 119s 3s/step - loss: 0.5359 - accuracy: 0.7253 - val_loss: 0.6969 - val_accuracy: 0.6304
Epoch 4/30
47/47 [==============================] - 135s 3s/step - loss: 0.4222 - accuracy: 0.8088 - val_loss: 0.8028 - val_accuracy: 0.5435
Epoch 5/30
47/47 [==============================] - 165s 3s/step - loss: 0.3132 - accuracy: 0.8590 - val_loss: 0.7913 - val_accuracy: 0.6304
Epoch 6/30
47/47 [==============================] - 204s 4s/step - loss: 0.2049 - accuracy: 0.9318 - val_loss: 1.1957 - val_accuracy: 0.5435
Epoch 7/30
47/47 [==============================] - 182s 4s/step - loss: 0.1233 - accuracy: 0.9606 - val_loss: 1.0797 - val_accuracy: 0.6087
Epoch 8/30
47/47 [========

KeyboardInterrupt: 

In [7]:
#resnet = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(HEIGHT,WIDTH,3), classes=1, classifier_activation='sigmoid')
resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(HEIGHT,WIDTH,3), classes=1, classifier_activation='sigmoid')
#resnet = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(HEIGHT,WIDTH,3), classes=1)
resnet.trainable = True
model = Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-19 21:52:23.107661: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-19 21:52:23.116965: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
training = model.fit(X_train, y_train, batch_size = 32, validation_split=0.2, epochs = 20)

Epoch 1/20


2022-08-21 15:37:08.392297: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-21 15:37:09.639810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 847.2183 - accuracy: 0.5230

2022-08-21 15:39:01.098611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 125s 5s/step - loss: 847.2183 - accuracy: 0.5230 - val_loss: 1.5443 - val_accuracy: 0.5602
Epoch 2/20
24/24 [==============================] - 135s 6s/step - loss: 0.7684 - accuracy: 0.5059 - val_loss: 0.7006 - val_accuracy: 0.4031
Epoch 3/20
24/24 [==============================] - 95s 4s/step - loss: 0.6925 - accuracy: 0.5243 - val_loss: 0.7047 - val_accuracy: 0.4031
Epoch 4/20
24/24 [==============================] - 101s 4s/step - loss: 0.6931 - accuracy: 0.5243 - val_loss: 0.6977 - val_accuracy: 0.4031
Epoch 5/20
19/24 [======================>.......] - ETA: 20s - loss: 0.6979 - accuracy: 0.5082

KeyboardInterrupt: 

In [ ]:
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.ylim([0.5,1.])
plt.show()

NameError: name 'training' is not defined

In [1]:
y_predd = model.predict(X_test)
y_pred = (y_predd>0.5).astype(int)

# Plotting the confusing matrix
mat = confusion_matrix(y_test, y_pred.round())
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')
print('accuracy: ' , accuracy_score(y_test, y_pred.round()).round(2))
print('recall: ' , recall_score(y_test, y_pred.round()).round(2))
print('precision: ' , precision_score(y_test, y_pred.round()).round(2))

NameError: name 'model' is not defined

In [ ]:
y_predd

array([[2.68996274e-08],
       [1.39304657e-08],
       [2.87037061e-08],
       [4.34488676e-08],
       [3.32823440e-08],
       [2.52988173e-08],
       [1.49417989e-08],
       [2.99615692e-08],
       [1.61255102e-07],
       [2.13042526e-08],
       [3.50035059e-08],
       [1.84442285e-07],
       [5.24639816e-08],
       [7.54871792e-08],
       [2.15951736e-08],
       [5.28669872e-08],
       [7.56923413e-08],
       [6.54435652e-08],
       [3.34824932e-08],
       [2.36862796e-08],
       [1.20008153e-07],
       [4.09835330e-08],
       [4.35110650e-08],
       [8.05706506e-08],
       [5.11284561e-08],
       [6.92053504e-07],
       [1.01437454e-08],
       [1.04250306e-08],
       [1.98675529e-08],
       [1.15732412e-07],
       [3.38229817e-08],
       [6.54294681e-08],
       [2.25887202e-08],
       [5.67087319e-08],
       [2.98763716e-08],
       [1.37964848e-08],
       [6.91550923e-08],
       [9.61344355e-08],
       [1.19023937e-08],
       [3.01369276e-08],
